In [1]:
import json
import nltk
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity
import random
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    return tokens


In [8]:
def preprocess_dataset(file_path):
    with open('magnus4.json', 'r') as file:
        data = json.load(file)

    questions = []
    answers = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            # Preprocess question text
            question_tokens = preprocess_text(pattern)
            questions.append(' '.join(question_tokens))
            answers.append(intent["responses"])

    return questions, answers

In [9]:
def encode_sentences(sentences):
    # Load Universal Sentence Encoder model
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

    # Encode sentences into embeddings
    embeddings = embed(sentences)
    return embeddings.numpy()

In [10]:
def get_most_similar_question(input_question, questions, embeddings):
    input_embedding = encode_sentences([input_question])[0]

    similarity_scores = cosine_similarity([input_embedding], embeddings)[0]

    most_similar_index = similarity_scores.argmax()
    return questions[most_similar_index]


In [11]:
def get_random_reply(replies):
  return random.choice(replies)

In [12]:
file_path = 'dataset.json'
questions, answers = preprocess_dataset(file_path)
embeddings = encode_sentences(questions)
#encode the set of questions

print("ChatBot: Hello! How can I assist you today?")

while True:
    input_question = input("User: ")

    if input_question.lower() in ['exit', 'quit','done']:
        print("ChatBot: Goodbye! Have a great day...")
        break

    most_similar_question = get_most_similar_question(input_question, questions, embeddings)
    index = questions.index(most_similar_question)
    response = get_random_reply(answers[index])

    print("ChatBot:", response)

ChatBot: Hello! How can I assist you today?
User: What all colors are available for magnusEX
ChatBot: The Magnus EX comes in 5 iconic colors- Metallic Red, Galactic Grey, Graphite Black, Glacial White and Ocean Blue
User: What is the cost?
ChatBot: Enter your State
User: assam
ChatBot: Rs 106,900
User: Done
ChatBot: Goodbye! Have a great day...
